# **Step1: Install amd import libraries**

In [ ]:
!pip install -qq "transformers==4.35" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1" "tiktoken" "rouge-score"
import pandas as pd
import torch
from datasets import Dataset, load_dataset
from random import randrange
from peft import LoraConfig, get_peft_model, AutoPeftModelForCausalLM
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import torch
from peft import AutoPeftModelForCausalLM
import transformers
from tqdm import tqdm
import json

print(transformers.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device ',device,' is being used')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 56.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.6 M

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


4.35.0
Device  cuda  is being used


# **Step 2: Data Loading**

In [ ]:
# Dataset
test_dataset = load_dataset("eli5_category",split="test")
number_of_questions_to_get_score = 1000
test_dataset


Extracting data files:   0%|          | 0/4 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/91772 [00:00<?, ? examples/s]

Generating validation1 split:   0%|          | 0/5446 [00:00<?, ? examples/s]

Generating validation2 split:   0%|          | 0/2375 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5411 [00:00<?, ? examples/s]

Dataset({
    features: ['q_id', 'title', 'selftext', 'category', 'subreddit', 'answers', 'title_urls', 'selftext_urls'],
    num_rows: 5411
})

# **Step 3: Load Saved Model**

In [ ]:
! gdown 1l_tYk9wnb9t58v5o__Qbm9RJt7mbjRtx
print(' If you cannot download dataset: please use this link:\n https://drive.google.com/file/d/1l_tYk9wnb9t58v5o__Qbm9RJt7mbjRtx/view?usp=sharing')
!unzip -qq '/content/Final_Laama_Model.zip'

Downloading...
From (original): https://drive.google.com/uc?id=1l_tYk9wnb9t58v5o__Qbm9RJt7mbjRtx
From (redirected): https://drive.google.com/uc?id=1l_tYk9wnb9t58v5o__Qbm9RJt7mbjRtx&confirm=t&uuid=f4f5a42a-5b1a-4c0d-8396-201e46a72cc9
To: /content/ELI5_Model.zip
100% 493M/493M [00:07<00:00, 65.6MB/s]


In [ ]:
model_directory = '/content/results/lamma/checkpoint-2100' # Set this to your directory

new_model = AutoPeftModelForCausalLM.from_pretrained(
    model_directory,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## 3-1 Load tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_directory, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# **Step 4: Generate answer**

In [ ]:
def answer_query(query,temperature=0.6,long_answer = False):

    if long_answer:
      prompt= 'Generatee long answer for this question ' + query

    else:
      prompt= 'Generate short answer for this question ' + query

    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()

    if long_answer:

        outputs = new_model.generate(input_ids=input_ids,


                             min_length=100,
                             max_new_tokens=300,
                                 #repetition_penalty=0.2,
                            #  do_sample=True,
                            #  top_p=0.9,
                             temperature=temperature)

    else:

        outputs = new_model.generate(input_ids=input_ids,


                             min_length=30,
                             max_length = 100,
                             max_new_tokens=70,
                                 #repetition_penalty=0.2,
                            #  do_sample=True,
                            #  top_p=0.9,
                             temperature=temperature)
    answer = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]

    return answer

## 4-1 Make answer pretty

In [ ]:
# Remove question from answer and remove last uncomplete sentence
def pretty_asnwer(answer):
    new_answer = answer
    if new_answer.endswith('.') or new_answer.endswith('!') or new_answer.endswith('?'):

        last_index_of_question = len(prompt) +41
        new_answer = new_answer[last_index_of_question:]
        if new_answer.find('Answer')>-1:
          index = new_answer.find('Answer')
          new_answer = new_answer[index+8:]
        #print(new_answer)

    else:
        #print('Not complete answer')
        last_index_of_question = len(prompt) +41
        new_answer = new_answer[last_index_of_question:]
        last_sentence = new_answer.split('. ')[-1]
        new_answer = new_answer.replace(last_sentence, " ")
        if new_answer.find('Answer')>-1:
          #print('here')
          index = new_answer.find('Answer')
          new_answer = new_answer[index+8:]



    return new_answer



# **Step 5: Create a dataset including generated answers**

In [ ]:
generated_answer_list = []

disablity_of_model = 0

for prompt in tqdm(test_dataset['title'][:number_of_questions_to_get_score]):
  #print('index: ', index)

  answer = answer_query(prompt,long_answer=True)  # Generate Answer
  new_answer = pretty_asnwer(answer) # Make it beatiful

  j=0
  while len(new_answer)<5: # Generate again if, last time model was disable
      #print(f'index_{index}_J{j}:')
      disablity_of_model+=1
      answer = answer_query(prompt,long_answer=True)
      new_answer = pretty_asnwer(answer)
      #print('new_answer: ',new_answer)

  generated_answer_list.append(new_answer) # Store answer




# Create a dataframe with orginal questions and long genereated answers
dictionary = {'questions':test_dataset['title'][:number_of_questions_to_get_score],'Generated_answers':generated_answer_list}
generated_dataset = pd.DataFrame(dictionary)
print('disablity_of_model: ',disablity_of_model)

100%|██████████| 1000/1000 [4:03:15<00:00, 14.60s/it]

disablity_of_model:  41


## 5-1 Add true answers to dataset

In [ ]:
# Add True answers to dataset
concatenaed_answer_list = []
for answer_list in test_dataset['answers'][:number_of_questions_to_get_score]:
  concatenaed_answer = ' '.join(answer_list['text'])
  concatenaed_answer_list.append(concatenaed_answer)

generated_dataset['True_answers'] = concatenaed_answer_list
generated_dataset

,questions,Generated_answers,True_answers
0,Why do old games running on new hardware still...,I'm an old game dev. I've seen a lot of this s...,The XBox is emulating NES hardware and running...
1,Why are electrical lines slack between poles?,Because the weight of the wire is enough to pu...,"Because if they were pulled taut, then a stron..."
2,It's been 12 years since the tallest roller co...,"It's not a limit of the roller coaster, it's a...",It's really about cost and safety. Look at the...
3,What exactly IS a belt drive motor?,A belt drive motor is a motor that uses a belt...,A motor with a pulley and a belt which drives ...
4,"What is this ""Split Bolt"" for and why is it 15...",...it's an old-fashioned socket wrench with a ...,"Everybody else is right as to what it's for, m..."
...,...,...,...
995,Why do so many passenger planes use the Deakin...,The Deakin method is used for a couple of reas...,"Noise abatement, mostly. Long, slow climbs ove..."
996,When locksmiths started producing keys on a la...,( URL_0 ) & #x200B; > I'm not sure I understan...,Locks don't *need* to be totally unique. If yo...
997,How solar panels work,"Solar panels are made of photovoltaic cells, w...",2 layers of silicon crystals are laid on top o...
998,How do Maglev trains compare to regular trains...,x200B; Maglev trains use magnetic levitation...,Maglev (magnetic levitation) is needed if you ...


# **Step 6: Compute Rouge Metrics**

In [ ]:

from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

def calculate_rouge(predicted, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
    scores = scorer.score(reference, predicted)
    scores = {metric: {sub_metric: round(value * 100, 2) for sub_metric, value in scores[metric]._asdict().items()} for metric in scores}

    return scores



rouge_list = []
rouge1_scores = {'precision': [], 'recall': [], 'fmeasure': []}
rouge2_scores = {'precision': [], 'recall': [], 'fmeasure': []}
rougeL_scores = {'precision': [], 'recall': [], 'fmeasure': []}

for index in range(generated_dataset.shape[0]):
  predicted_text = generated_dataset.iloc[index,1]
  reference_text = generated_dataset.iloc[index,2]

  rouge_scores = calculate_rouge(predicted_text, reference_text)
  rouge_list.append(rouge_scores)

  for sub_metric in rouge1_scores:
        rouge1_scores[sub_metric].append(rouge_scores['rouge1'][sub_metric])
        rouge2_scores[sub_metric].append(rouge_scores['rouge2'][sub_metric])
        rougeL_scores[sub_metric].append(rouge_scores['rougeL'][sub_metric])


generated_dataset['Rouge Scores'] = rouge_list


#mean
mean_rouge1_scores = {sub_metric: round(sum(rouge1_scores[sub_metric]) / len(rouge1_scores[sub_metric]), 2) for sub_metric in rouge1_scores}
mean_rouge2_scores = {sub_metric: round(sum(rouge2_scores[sub_metric]) / len(rouge2_scores[sub_metric]), 2) for sub_metric in rouge2_scores}
mean_rougeL_scores = {sub_metric: round(sum(rougeL_scores[sub_metric]) / len(rougeL_scores[sub_metric]), 2) for sub_metric in rougeL_scores}
print('For 1000 items of test dataset:')
print('*'*20)
print("Mean ROUGE-1 scores (as percentages):", mean_rouge1_scores)
print("Mean ROUGE-2 scores (as percentages):", mean_rouge2_scores)
print("Mean ROUGE-L scores (as percentages):", mean_rougeL_scores)

For 1000 items of test dataset:
********************
Mean ROUGE-1 scores (as percentages): {'precision': 37.14, 'recall': 24.35, 'fmeasure': 24.24}
Mean ROUGE-2 scores (as percentages): {'precision': 7.41, 'recall': 4.18, 'fmeasure': 4.31}
Mean ROUGE-L scores (as percentages): {'precision': 21.3, 'recall': 14.59, 'fmeasure': 13.94}


# **Step 7: Save files**

In [ ]:

# Save dataframe
generated_dataset.to_csv(f"/content/generated_{number_of_questions_to_get_score}_test_dataset.csv",sep=',',index=False)
# Svae mean scores
# Filename
filename = f"/content/mean_rouge_scores_for_{number_of_questions_to_get_score}_test_dataset.txt"
note = f'These valeus are for train_dataset of {number_of_questions_to_get_score} questions '
# Writing to the text file
with open(filename, 'w') as file:
    file.write(note + "\n\n")

    file.write("Mean ROUGE-1 scores:\n")
    file.write(json.dumps(mean_rouge1_scores, indent=4) + "\n\n")

    file.write("Mean ROUGE-2 scores:\n")
    file.write(json.dumps(mean_rouge2_scores, indent=4) + "\n\n")

    file.write("Mean ROUGE-L scores:\n")
    file.write(json.dumps(mean_rougeL_scores, indent=4) + "\n")

print(f"Data saved to {filename}")


Data saved to /kaggle/working/mean_rouge_scores_for_1000_test_dataset.txt
